<a href="https://colab.research.google.com/github/Sabelz/Gaussian_Processes_Inducing_Points/blob/main/utils/Models.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# The basic structure of all Gaussian process models

# Imports

In [7]:
from google.colab import drive
drive.mount('/content/drive')
%cd /content/drive/MyDrive/Master_Thesis_Alexander/
!git config --global user.email "alexander.sabelstrom.1040@student.uu.se"
!git config --global user.name "Sabelz"

import numpy as np
import matplotlib.pyplot as plt
import torch
!pip install gpytorch
import gpytorch
print("----------------------------------------------------------")
print("ALL MODELS: ")

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).
/content/drive/MyDrive/Master_Thesis_Alexander
----------------------------------------------------------
ALL MODELS: 


# Structured Kernel Interpollation (SKI/KISS-GP)   
https://docs.gpytorch.ai/en/latest/examples/02_Scalable_Exact_GPs/KISSGP_Regression.html

## 1D-4D data

In [12]:
class KISSGP(gpytorch.models.ExactGP):
    def __init__(self, train_x, train_y, likelihood, kernel):
        super(KISSGP, self).__init__(train_x, train_y, likelihood)

        # SKI requires a grid size hyperparameter. This util can help with that.
        grid_size = gpytorch.utils.grid.choose_grid_size(train_x)

        self.mean_module = gpytorch.means.ConstantMean()
        self.covar_module = gpytorch.kernels.ScaleKernel(
            gpytorch.kernels.GridInterpolationKernel(
                kernel, grid_size=grid_size, num_dims=train_x.shape[1]# Get dimension of training data
            )
        )

    def forward(self, x):
        mean_x = self.mean_module(x)
        covar_x = self.covar_module(x)
        return gpytorch.distributions.MultivariateNormal(mean_x, covar_x)
print("KISS-GP For 1D-4D data:")
print("Example:")
print("   likelihood = gpytorch.likelihoods.GaussianLikelihood()")
print("   kernel = gpytorch.kernels.RBFKernel()")
print("   model = KISSGP(train_x, train_y, likelihood, kernel)")

KISS-GP For 1D-4D data:
Example:
   likelihood = gpytorch.likelihoods.GaussianLikelihood()
   kernel = gpytorch.kernels.RBFKernel()
   model = KISSGP(train_x, train_y, likelihood, kernel)


## For higher dimensional data

In [13]:
class KISSGP_NDim(gpytorch.models.ExactGP):
    def __init__(self, train_x, train_y, likelihood, kernel):
        super(KISSGP_NDim, self).__init__(train_x, train_y, likelihood)

        # SKI requires a grid size hyperparameter. This util can help with that
        # We're setting Kronecker structure to False because we're using an additive structure decomposition
        grid_size = gpytorch.utils.grid.choose_grid_size(train_x, kronecker_structure=False)

        self.mean_module = gpytorch.means.ConstantMean()
        self.covar_module = gpytorch.kernels.AdditiveStructureKernel(
            gpytorch.kernels.ScaleKernel(
                gpytorch.kernels.GridInterpolationKernel(
                    kernel, grid_size=128, num_dims=1
                )
            ), num_dims=train_x.shape[1]# Get dimension of training data
        )

    def forward(self, x):
        mean_x = self.mean_module(x)
        covar_x = self.covar_module(x)
        return gpytorch.distributions.MultivariateNormal(mean_x, covar_x)


print("KISS-GP For higher dimensional data:")
print("Example:")
print("   likelihood = gpytorch.likelihoods.GaussianLikelihood()")
print("   kernel = gpytorch.kernels.RBFKernel()")
print("   model = KISSGP_NDim(train_x, train_y, likelihood, kernel)")

KISS-GP For higher dimensional data:
Example:
   likelihood = gpytorch.likelihoods.GaussianLikelihood()
   kernel = gpytorch.kernels.RBFKernel()
   model = KISSGP_NDim(train_x, train_y, likelihood, kernel)
